In [31]:
# import statements
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import deque
import random
import importlib

import player
import rl_player
importlib.reload(player)
importlib.reload(rl_player)
from player import *
from rl_player import *

In [32]:
def train_agent(agent, env, num_episodes, batch_size, epsilon_decay):
    total_rewards = []
    epsilon = 1.0

    for episode in range(num_episodes):
        initial_state = env.reset()
        state = initial_state
        done = False
        total_reward = 0

        while not done:
            # Choose an action using epsilon-greedy strategy
            action = agent.get_action(state, agent.name, epsilon)

            # Take the chosen action and observe the next state, reward, and done flag from the environment
            next_state, reward, done = env.step(action)

            # Add the experience to the agent's replay buffer
            agent.add_experience(state, action, reward, next_state, done)

            # Update the agent's Q-network if there are enough experiences in the replay buffer
            if len(agent.replay_buffer) >= batch_size:
                agent.replay_experience(batch_size, agent.name)

            # Update the game state and history for the next iteration
            state = next_state

            # Accumulate the total reward
            total_reward += reward

        # Decay epsilon
        epsilon *= epsilon_decay

        # Save the total reward for this episode
        total_rewards.append(total_reward)

        # Print the episode number and total reward
        print(f"Episode {episode + 1}/{num_episodes} - Total Reward: {total_reward}")

    return total_rewards

In [33]:
state_dim = 43
action_dim = 10
learning_rate = 0.0001
gamma = 0.99

agent = QLearningAgent(state_dim, action_dim, learning_rate, gamma, 'Player 1')
env = Environment('Player 1')

num_episodes = 1000
batch_size = 32
epsilon_decay = 0.99

total_rewards = train_agent(agent, env, num_episodes, batch_size, epsilon_decay)

Episode 1/1000 - Total Reward: 0
Episode 2/1000 - Total Reward: 0
Episode 3/1000 - Total Reward: 0
Episode 4/1000 - Total Reward: 0
Episode 5/1000 - Total Reward: 0
Episode 6/1000 - Total Reward: 0
Episode 7/1000 - Total Reward: 0
Episode 8/1000 - Total Reward: 0
Episode 9/1000 - Total Reward: 0
Episode 10/1000 - Total Reward: 0
Episode 11/1000 - Total Reward: 0
Episode 12/1000 - Total Reward: 0
Episode 13/1000 - Total Reward: 0
Episode 14/1000 - Total Reward: 0
Episode 15/1000 - Total Reward: 0
Episode 16/1000 - Total Reward: 0
Episode 17/1000 - Total Reward: 0
Episode 18/1000 - Total Reward: 0
Episode 19/1000 - Total Reward: 0
Episode 20/1000 - Total Reward: 0
Episode 21/1000 - Total Reward: 0
Episode 22/1000 - Total Reward: 0
Episode 23/1000 - Total Reward: 0
Episode 24/1000 - Total Reward: 0
Episode 25/1000 - Total Reward: 0
Episode 26/1000 - Total Reward: 0
Episode 27/1000 - Total Reward: 0
Episode 28/1000 - Total Reward: 0
Episode 29/1000 - Total Reward: 0
Episode 30/1000 - Total

IndexError: tensors used as indices must be long, int, byte or bool tensors